In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors

from numpy import array
from numpy import max
import numpy as np
import pandas as pd
import math
import folium

In [2]:
from download import download

# We choose daily data
url = "https://opendata.arcgis.com/datasets/2ab16a5fb61f42c1a689fd9cc466383f_0.csv"
path_target = "datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv"
download(url, path_target, replace=False)

Replace is False and data exists, so doing nothing. Use replace==True to re-download the data.


'datasets/Mesure_journaliere_Region_Occitanie_Polluants_Principaux.csv'

## 1) Data treatment

In [3]:
occ_df = pd.read_csv(path_target)
occ_df = occ_df[occ_df['nom_poll'] == 'O3'] # only ozone

Que veut-on représenter? 
   +  Toutes les villes de notre data set en Occitanie
   +  La question est quand ?
   + Peut on faire varier la date interactivement
   + Voila pourquoi on veut une video, on va voire la pollution évoluer
   
J'ai reflechi cette nuit et on developpera ici:
   + Une map de l'occitanie qui affiche les données journalières
   + Elle sera interactive et on pourra choisir le jour qu'on veut
   + Il faudra certainement et au préalable nettoyer nos données

In [4]:
# On va traiter les données afin de rajouter une colonne de la date (jour)
occ_df['jour'] = pd.to_datetime(occ_df['date_debut']).dt.to_period('D')
# On rajoute une colonne pour avoir des valeurs standardisées
# Cela va servir à avoir une échelle des couleurs plus claire
occ_df['centrer_reduire'] = (occ_df[['valeur']] - np.mean(occ_df[['valeur']]))/ np.std(occ_df[['valeur']])

C:\Users\megan\Anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:1269: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [5]:
occ_df.nom_station.unique()

array(['Mende - Vernede urbain', 'Saze - rural',
       'Toulouse-Berthelot Urbain', 'Montpellier Nord - Périurbain',
       'Toulouse-Mazades Urbain', 'Albi Urbain',
       'Montpellier - Prés d Arènes Urbain', 'Toulouse-Jacquier Urbain',
       'Lourdes-Lapaca Urbain', 'Castres-Travet Urbain',
       'Tarbes-Dupuy Urbain', 'Nîmes Sud - urbain',
       'La Calmette - Périurbain', 'Peyrusse Vieille Rural',
       'Montgiscard Urbain', 'Belesta en Lauragais Rural',
       'Montpellier Sud - Périurbain', 'Saint Estève - périurbain',
       'Agathois-Piscénois - Périurbain',
       'Biterrois-Narbonnais - Périurbain',
       'Saint Gaudens-Miramont Industriel', 'Rodez Urbain',
       'Caunes-Minervois - rural', 'Carcassonne - urbain',
       'Station Pedagogique', 'Perpignan Centre - urbain',
       'Roqueredonde - rurale', 'Biars-sur-Cere - urbain',
       'Vallabrègues - rural'], dtype=object)

In [6]:
occ_df.jour.unique() # 362 jours, on fera varier notre map selon ces jours.

<PeriodArray>
['2019-12-27', '2019-12-28', '2020-02-19', '2020-02-26', '2019-12-29',
 '2019-12-30', '2020-02-20', '2019-12-31', '2020-01-01', '2019-10-16',
 ...
 '2020-04-07', '2020-04-08', '2020-03-27', '2020-03-28', '2020-03-29',
 '2020-03-30', '2020-03-31', '2020-04-01', '2019-12-22', '2019-12-25']
Length: 363, dtype: period[D]

In [10]:
import branca.colormap as cm

linear = cm.LinearColormap(
    ['green', 'yellow', 'red'],
    vmin=-3, vmax=1
)
# couleurs

In [12]:
from ipywidgets import interact  # widget manipulation
from IPython.display import HTML

In [13]:
# rajoute Paris !!! 

def interactive_map(day = '2020-03-26'):
    
    occ_map = occ_df[occ_df['jour'] == day]
    
    map_int = folium.Map(location = [43.9333, 2.15], 
                         zoom_start = 7.5, 
                         tiles = 'Stamen Terrain')
    
    for i in range(0,len(occ_map)):
        folium.Circle(
            location = [occ_map.iloc[i]['Y'], occ_map.iloc[i]['X']],
            popup = occ_map.iloc[i]['nom_station'],
            radius = occ_map.iloc[i]['valeur']*100,
            color = 'black',
            fill = True,
            fill_color = linear(occ_map.iloc[i]['centrer_reduire']),
            fill_opacity = 0.5,
            opacity = 0.4,
        ).add_to(map_int)
    
    return(map_int)

In [14]:
interact(interactive_map, month=occ_df.jour.unique())

interactive(children=(Text(value='2020-03-26', description='day'), Output()), _dom_classes=('widget-interact',…

<function __main__.interactive_map(day='2020-03-26')>

## 4) Animation 


In [15]:
import matplotlib.animation as animation
# Comment obtient on une image de ce truc ?
# Animons tout ça...